In [1]:
import tensorflow as tf
import os 
import logging
from functions import load_train_data

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
logging.basicConfig(level = logging.INFO)
gpus = tf.config.experimental.list_physical_devices('GPU')
try:
    tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(
    gpus[1],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=40000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
except RuntimeError as e:
    logging.info(e)

In [2]:
#3flavor_poisson, nsi_poisson
learn_target = 'nsi_poisson'

x_train, y_train, x_val, y_val = load_train_data(learn_target)

In [3]:
def create_dnn_model():
    inputs = tf.keras.Input(shape=(len(x_train[0]),), name = 'input')
    features = tf.keras.layers.BatchNormalization()(inputs)
    features = tf.keras.layers.Dense(256, activation='relu')(features)
    features = tf.keras.layers.Dense(256, activation='relu')(features)
    features = tf.keras.layers.Dense(64, activation='relu')(features)
    features = tf.keras.layers.Dense(len(y_train[0]), activation='linear')(features)
    outputs = features
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

In [4]:
dnn = create_dnn_model()
dnn.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.MeanSquaredError()],
)

In [6]:
dnn.fit(x_train, y_train,
           validation_data = (x_val, y_val),
           batch_size=1024,
           epochs=100,
           verbose=1,
           shuffle = True
         )

Epoch 1/100
8790/8790 [==============================] - 36s 4ms/step - loss: 0.0449 - mean_squared_error: 0.0449 - val_loss: 0.0326 - val_mean_squared_error: 0.0326
Epoch 2/100
8790/8790 [==============================] - 33s 4ms/step - loss: 0.0301 - mean_squared_error: 0.0301 - val_loss: 0.0275 - val_mean_squared_error: 0.0275
Epoch 3/100
8790/8790 [==============================] - 32s 4ms/step - loss: 0.0277 - mean_squared_error: 0.0277 - val_loss: 0.0270 - val_mean_squared_error: 0.0270
Epoch 4/100
8790/8790 [==============================] - 32s 4ms/step - loss: 0.0266 - mean_squared_error: 0.0266 - val_loss: 0.0247 - val_mean_squared_error: 0.0247
Epoch 5/100
8790/8790 [==============================] - 32s 4ms/step - loss: 0.0258 - mean_squared_error: 0.0258 - val_loss: 0.0237 - val_mean_squared_error: 0.0237
Epoch 6/100
8790/8790 [==============================] - 32s 4ms/step - loss: 0.0253 - mean_squared_error: 0.0253 - val_loss: 0.0240 - val_mean_squared_error: 0.0240
Epoc

In [7]:
index = 1
while os.path.isfile('./dnn/' + learn_target + '/{}.h5'.format(index)):
    index += 1
dnn.save('./dnn/' + learn_target + '/{}.h5'.format(index))